# Develop a GNN model for ADMET prediction

## Analyse the TDC Dataset

In [1]:
try:
  from tdc.benchmark_group import admet_group
except:
  !pip install pytdc
  from tdc.benchmark_group import admet_group

In [2]:
try:
  from modules.utils import set_device, set_seed
except:
  !git clone https://github.com/Huvinesh-Rajendran-12/admet.git
  !mv admet/modules 
  !rm -rf admet/
  from modules.utils import set_device, set_seed

In [3]:
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    import torchvision
    from torch_geometric.data import Data, DataLoader
    from torch_geometric.nn import GCNConv, global_mean_pool
    from torch.utils.data import Subset
    from rdkit import Chem
    from rdkit.Chem import AllChem
    import logging 
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    !pip3 install torch_geometric rdkit
    import torch
    from torch_geometric.data import Data, DataLoader
    from torch_geometric.nn import GCNConv, global_mean_pool
    from rdkit import Chem
    from rdkit.Chem import AllChem
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
pyenv: version `3.13' is not installed (set by /Users/huvinesh/Documents/admet/.python-version)
pyenv: version `3.13' is not installed (set by /Users/huvinesh/Documents/admet/.python-version)
torch version: 2.6.0
torchvision version: 0.21.0


In [4]:
from modules.data.config import ADMETConfig
from modules.data.dataset import LazyADMETDataset, ADMETDataset

config = ADMETConfig()

In [5]:
config

ADMETConfig(root_dir='data', cache_file='processed_graphs.pkl', add_hydrogens=True, calculate_charges=True, use_3d_coordinates=False, augment_data=False, cache_graphs=True, n_jobs=4, batch_size=100, atom_types=['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'I', 'B', 'Na', 'K', 'Li', 'Ca'], skip_elements=['Al', 'Fe', 'Zn', 'Mg', 'Cu', 'Na', 'K', 'Li', 'Ca', 'Ba', 'Sr', 'Cs', 'Rb', 'Be', 'Ra', 'Hg', 'Cd', 'Pb', 'Mn', 'Co', 'Ni', 'Zr', 'Cr', 'Mo', 'W', 'V', 'Nb', 'Ta', 'Ti', 'Hf', 'Re', 'Os', 'Ir', 'Pt', 'Pd', 'Rh', 'Ru', 'Au', 'Ag', 'Bi', 'Sb', 'As', 'Sn', 'Pb', 'Te', 'Se', 'Si', 'B', 'Ga', 'In', 'Tl', 'Ge', 'P', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Y', 'Sc', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu'], split_ratio=0.8, random_seed=42, log_level='INFO')

In [6]:
logging.basicConfig(
    level=getattr(logging, config.log_level.upper()),
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

logger.info("Starting ADMET data processing")

dataset = LazyADMETDataset(config)

train_indices, test_indices = dataset.get_split_indices()

val_indices = train_indices[:5000]

train_data = Subset(dataset, train_indices)
val_data = Subset(dataset, val_indices)
test_data = Subset(dataset, test_indices)
# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Get task information
task_info = dataset.get_task_info()

logger.info(f"Dataset loaded with {len(dataset)} molecules")
logger.info(f"Number of tasks: {len(task_info['task_names'])}")
logger.info(f"Training set size: {len(train_indices)}")
logger.info(f"Test set size: {len(test_indices)}")

# Print some task information
for i, task_name in enumerate(task_info['task_names']):
    task_type = task_info['task_types'][i]
    logger.info(f"Task {i}: {task_name} ({task_type})")
    
    if task_type == 'regression':
        logger.info(f"  Mean: {task_info['task_means'].get(i, 'N/A')}")
        logger.info(f"  Std: {task_info['task_stds'].get(i, 'N/A')}")

# Example of iterating through batches
logger.info("Checking first batch from training loader")
for batch in train_loader:
    logger.info(f"Batch size: {batch.num_graphs}")
    logger.info(f"Features shape: {batch.x.shape}")
    logger.info(f"Edge index shape: {batch.edge_index.shape}")
    logger.info(f"Edge attr shape: {batch.edge_attr.shape}")
    logger.info(f"Target shape: {batch.y.shape}")
    break

logger.info("ADMET data processing complete")

2025-02-28 14:10:46 - __main__ - INFO - Starting ADMET data processing
2025-02-28 14:10:46 - modules.data.dataset - INFO - Loading ADMET dataset metadata
Found local copy...
2025-02-28 14:10:46 - modules.data.dataset - INFO - Loading metadata for dataset: caco2_wang (728 molecules)
2025-02-28 14:10:46 - modules.data.dataset - INFO - Loading metadata for dataset: hia_hou (461 molecules)
2025-02-28 14:10:46 - modules.data.dataset - INFO - Loading metadata for dataset: pgp_broccatelli (973 molecules)
2025-02-28 14:10:46 - modules.data.dataset - INFO - Loading metadata for dataset: bioavailability_ma (512 molecules)
2025-02-28 14:10:46 - modules.data.dataset - INFO - Loading metadata for dataset: lipophilicity_astrazeneca (3360 molecules)
2025-02-28 14:10:46 - modules.data.dataset - INFO - Loading metadata for dataset: solubility_aqsoldb (7985 molecules)
2025-02-28 14:10:47 - modules.data.dataset - INFO - Loading metadata for dataset: bbb_martins (1624 molecules)
2025-02-28 14:10:47 - modu

In [7]:
len(dataset.smiles_list)

65430

In [8]:
def get_model_dimensions(dataset):
    """
    Determine node and edge feature dimensions from the dataset
    
    Args:
        dataset: ADMET dataset
        
    Returns:
        Tuple of (node_features, edge_features)
    """
    # Try to get dimensions from the first data point
    try:
        # For LazyADMETDataset, extract dimensions from the first molecule
        data = dataset[0]
        
        # Get feature dimensions
        node_features = data.x.size(1)
        
        # Edge features might be empty if no bonds
        if data.edge_attr.size(0) > 0:
            edge_features = data.edge_attr.size(1)
        else:
            # Check another molecule if possible
            for i in range(1, min(10, len(dataset))):
                data = dataset[i]
                if data.edge_attr.size(0) > 0:
                    edge_features = data.edge_attr.size(1)
                    break
            else:
                # Default if no molecules with bonds found
                edge_features = 7  # Default bond feature size
        
        return node_features, edge_features
    
    except Exception as e:
        logging.warning(f"Error determining dimensions: {e}")
        # Default values if automatic determination fails
        return 32, 7

In [9]:
node_features, edge_features = get_model_dimensions(dataset)
node_features, edge_features

(32, 7)

In [10]:
from modules.gnn_model import ADMETPredictor

model = ADMETPredictor(
    node_features=node_features,
    edge_features=edge_features,
    hidden_channels=128,
    num_layers=3,
    dropout_rate=0.3,
    task_types=task_info['task_types'],
    use_edge_features=True,
    gnn_type='GIN',
    pool_type='mean'
)

In [11]:
model

ADMETPredictor(
  (gnn_layers): ModuleList(
    (0): GINConv(nn=Sequential(
      (0): Linear(in_features=32, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
    ))
    (1-2): 2 x GINConv(nn=Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
    ))
  )
  (batch_norms): ModuleList(
    (0-2): 3 x BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (task_heads): ModuleList(
    (0-21): 22 x Sequential(
      (0): Linear(in_features=128, out_features=64, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.3, inplace=False)
      (3): Linear(in_features=64, out_features=1, bias=True)
    )
  )
)

In [17]:
from modules.trainer import Trainer

trainer = Trainer(
    model=model,
    dataset=dataset,
    train_indices=train_indices,
    val_indices=val_indices,
    test_indices=test_indices,
    device='cpu'
)

In [18]:
trainer.device

'cpu'

In [19]:
trainer.train_epoch()

Training:   0%|          | 1/1635 [00:00<04:39,  5.85it/s, loss=nan]2025-02-28 14:13:30 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: CCCCCCCCCCCC(=O)[O-].CCCCCCCCCCCC(=O)[O-].CCCC[Sn+2]CCCC
2025-02-28 14:13:30 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: COC(=O)[C@@]1(Cc2ccc(OC)cc2)[C@H]2c3cc(C(=O)N(C)C)n(Cc4ccsc4Br)c3C[C@H]2CN1C(=O)c1ccccc1
Training:   0%|          | 1/1635 [00:00<04:39,  5.85it/s, loss=nan]2025-02-28 14:13:30 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: CN(C)CCCSc1nc2ccccc2[nH]1
2025-02-28 14:13:30 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: C[C@@H](CN(C)C)CN1c2ccccc2Sc2ccccc21.C[C@@H](CN(C)C)CN1c2ccccc2Sc2ccccc21.O=C(O)[C@@H](O)[C@@H](O)C(=O)O
2025-02-28 14:13:30 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: O=[N+]([O-])c1ccc(OP(=O)(OCCCCl)OCCCCl)cc1


[Data(x=[62, 32], edge_index=[2, 128], edge_attr=[128, 7], smiles='CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1[C@H](C(=O)O)C[C@@H]2CCC[C@@H]21', task_id=[1], task_name='bioavailability_ma', task_type='classification', y=[1])]
tensor([[nan]], grad_fn=<AddmmBackward0>)
tensor([[1.]])
[Data(x=[64, 32], edge_index=[2, 134], edge_attr=[134, 7], smiles='CC(=O)Nc1cccc(Nc2ncnc(N3CCC(OCc4ccc(OC(F)(F)F)cc4)CC3)n2)c1C', task_id=[1], task_name='lipophilicity_astrazeneca', task_type='regression', y=[1]), Data(x=[55, 32], edge_index=[2, 118], edge_attr=[118, 7], smiles='C[C@@H](NC1=CC(=O)CC1)c1ccc(Nc2ncc3cc(-c4ccncc4)ccc3n2)cc1', task_id=[1], task_name='lipophilicity_astrazeneca', task_type='regression', y=[1])]
[Data(x=[35, 32], edge_index=[2, 68], edge_attr=[68, 7], smiles='CCCOP(=O)(OCCC)OCCC', task_id=[1], task_name='solubility_aqsoldb', task_type='regression', y=[1]), Data(x=[15, 32], edge_index=[2, 28], edge_attr=[28, 7], smiles='C=C(C)C(=O)OC', task_id=[1], task_name='solubility_aqsoldb', task

2025-02-28 14:13:30 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: Nc1nc2c(nc(Br)n2[C@@H]2O[C@H]3COP(=O)([O-])O[C@H]3[C@@H]2O)c(=O)[nH]1
Training:   0%|          | 3/1635 [00:00<03:10,  8.57it/s, loss=nan]2025-02-28 14:13:30 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: O=[N+]([O-])C1(Br)COCOC1
2025-02-28 14:13:30 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: CC(C)(c1ccccc1)c1ccc(O[P@]2OCC3(CO[P@@](Oc4ccc(C(C)(C)c5ccccc5)cc4C(C)(C)c4ccccc4)OC3)CO2)c(C(C)(C)c2ccccc2)c1


[Data(x=[65, 32], edge_index=[2, 138], edge_attr=[138, 7], smiles='Cc1c(CN2CCN(C(=O)[C@H](C)O)CC2)sc2c(N3CCOCC3)nc(-c3cnc(N)nc3)nc12', task_id=[1], task_name='lipophilicity_astrazeneca', task_type='regression', y=[1]), Data(x=[60, 32], edge_index=[2, 126], edge_attr=[126, 7], smiles='CC(=O)N[C@@H]1CCN(c2ccc(Nc3ncc(F)c(-c4cnc(C)n4C(C)C)n3)cc2)C1', task_id=[1], task_name='lipophilicity_astrazeneca', task_type='regression', y=[1])]
[Data(x=[20, 32], edge_index=[2, 38], edge_attr=[38, 7], smiles='CCCC(=O)OCC', task_id=[1], task_name='solubility_aqsoldb', task_type='regression', y=[1]), Data(x=[29, 32], edge_index=[2, 56], edge_attr=[56, 7], smiles='CCOC(=O)CCCC(=O)OCC', task_id=[1], task_name='solubility_aqsoldb', task_type='regression', y=[1]), Data(x=[30, 32], edge_index=[2, 60], edge_attr=[60, 7], smiles='CC(=O)CC(=O)Nc1ccc(C)cc1C', task_id=[1], task_name='solubility_aqsoldb', task_type='regression', y=[1]), Data(x=[40, 32], edge_index=[2, 84], edge_attr=[84, 7], smiles='CC(C)C1(C)N=C(c

Training:   0%|          | 5/1635 [00:00<02:58,  9.13it/s, loss=nan]2025-02-28 14:13:30 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: COC(=O)c1[nH]c2ccc(Br)cc2c1NC(=O)CN1CCN(C2CCCCC2)CC1
2025-02-28 14:13:30 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: O=c1[nH]c2cc(Br)cnc2nc1/C=C(\O)c1ccc(F)cc1
Training:   0%|          | 6/1635 [00:00<02:55,  9.27it/s, loss=nan]2025-02-28 14:13:30 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: COc1ccc([C@@H]2Sc3ccccc3N(CCN(C)C)C(=O)[C@@H]2OC(C)=O)cc1


...

Training:  10%|▉         | 160/1635 [00:23<04:30,  5.46it/s, loss=nan]

tensor([[nan]], grad_fn=<AddmmBackward0>)
tensor([[0.]])
[Data(x=[34, 32], edge_index=[2, 66], edge_attr=[66, 7], smiles='C=CC(C)(CCC=C(C)C)OC(C)=O', task_id=[1], task_name='ld50_zhu', task_type='regression', y=[1]), Data(x=[34, 32], edge_index=[2, 68], edge_attr=[68, 7], smiles='CCCCCCCc1ccccc1O', task_id=[1], task_name='ld50_zhu', task_type='regression', y=[1]), Data(x=[39, 32], edge_index=[2, 78], edge_attr=[78, 7], smiles='COCC(=O)N(C)C(=O)Oc1ccccc1OC(C)C', task_id=[1], task_name='ld50_zhu', task_type='regression', y=[1]), Data(x=[29, 32], edge_index=[2, 58], edge_attr=[58, 7], smiles='CCOP(=S)(CC)Sc1ccccc1', task_id=[1], task_name='ld50_zhu', task_type='regression', y=[1])]
[Data(x=[28, 32], edge_index=[2, 56], edge_attr=[56, 7], smiles='COc1ccc(CCN)cc1OC', task_id=[1], task_name='pgp_broccatelli', task_type='classification', y=[1])]
tensor([[nan]], grad_fn=<AddmmBackward0>)
tensor([[0.]])
[Data(x=[37, 32], edge_index=[2, 78], edge_attr=[78, 7], smiles='CN1CCC(C#N)(c2cccc3ccccc23)

2025-02-28 14:13:53 - modules.data.conversion - INFO - Skipping inorganic/metallic compound: CCCCCCCCCCCCCCCCCC(=O)[O-].CCCCCCCCCCCCCCCCCC(=O)[O-].[Ca+2]


[Data(x=[10, 32], edge_index=[2, 18], edge_attr=[18, 7], smiles='C=CC(N)=O', task_id=[1], task_name='caco2_wang', task_type='regression', y=[1])]
[Data(x=[33, 32], edge_index=[2, 66], edge_attr=[66, 7], smiles='CC(C)Cc1ccc(C(C)C(=O)O)cc1', task_id=[1], task_name='bioavailability_ma', task_type='classification', y=[1])]
tensor([[nan]], grad_fn=<AddmmBackward0>)
tensor([[1.]])
[Data(x=[46, 32], edge_index=[2, 94], edge_attr=[94, 7], smiles='CCS(=O)(=O)c1ccc(-c2cc(C(F)(F)F)ccc2CCC(=O)O)c(C)c1', task_id=[1], task_name='lipophilicity_astrazeneca', task_type='regression', y=[1])]
[Data(x=[30, 32], edge_index=[2, 64], edge_attr=[64, 7], smiles='O=C(O)c1ccc2c(c1C(=O)O)C(=O)c1ccccc1C2=O', task_id=[1], task_name='solubility_aqsoldb', task_type='regression', y=[1]), Data(x=[19, 32], edge_index=[2, 36], edge_attr=[36, 7], smiles='O=NN(CCO)CCO', task_id=[1], task_name='solubility_aqsoldb', task_type='regression', y=[1]), Data(x=[7, 32], edge_index=[2, 12], edge_attr=[12, 7], smiles='FS(F)(F)(F)(F)F

KeyboardInterrupt: 